In [1]:
import re
import pandas as pd
from sympy import false

text = "1879-03-14  1879年 3月14日 ドイツ帝国 ヴュルテンベルク王国 ウルム"

# 日付の抽出 (YYYY-MM-DD または YYYY年 MM月 DD日)
date_match = re.search(r"(\d{4}年\s*\d{1,2}月\s*\d{1,2}日)", text)
if date_match:
    date_str = date_match.group(1) or date_match.group(2)
else:
    date_str = None

# 出生地の抽出
place_match = re.search(r"(.+)", text)
if place_match:
    place_str = place_match.group(1)
    #日付を削除
    place_str = re.sub(r"(\d{4}-\d{2}-\d{2})|(\d{4}年\s*\d{1,2}月\d{1,2}日)", '', place_str).strip()
else:
    place_str = None


print(f"日付: {date_str}")
print(f"出生地: {place_str}")

日付: 1955年 4月18日
出生地: 76歳没 アメリカ合衆国 ニュージャージー州 プリンストン


In [12]:
import dateparser
# 日付のフォーマットを統一し、年、月、日に分解
if date_str:
    parsed_date = dateparser.parse(date_str, date_formats=['%Y-%m-%d', '%Y年%m月%d日'])
    if parsed_date:
      birth_year = parsed_date.year
      birth_month = parsed_date.month
      birth_day = parsed_date.day
    else:
      birth_year = None
      birth_month = None
      birth_day = None
else:
    birth_year = None
    birth_month = None
    birth_day = None

print(f"生年: {birth_year}")
print(f"生月: {birth_month}")
print(f"生日: {birth_day}")

生年: 1879
生月: 3
生日: 14


In [13]:
# 出生地の分解
if place_str:
    parts = place_str.split()
    if len(parts) >= 3:
      birth_country = parts[0]
      birth_kingdom = parts[1]
      birth_city = parts[2]
    elif len(parts) >= 2:
      birth_country = parts[0]
      birth_kingdom = None
      birth_city = parts[1]
    else:
      birth_country = None
      birth_kingdom = None
      birth_city = parts[0]

else:
    birth_country = None
    birth_kingdom = None
    birth_city = None

print(f"出生国: {birth_country}")
print(f"出生王国: {birth_kingdom}")
print(f"出生都市: {birth_city}")

出生国: ドイツ帝国
出生王国: ヴュルテンベルク王国
出生都市: ウルム


In [14]:
# DataFrame の作成
data = {
    "birth_year": [birth_year],
    "birth_month": [birth_month],
    "birth_day": [birth_day],
    "birth_country": [birth_country],
    "birth_kingdom": [birth_kingdom],
    "birth_city": [birth_city],
}
df = pd.DataFrame(data)

print(df)

   birth_year  birth_month  birth_day birth_country birth_kingdom birth_city
0        1879            3         14         ドイツ帝国    ヴュルテンベルク王国        ウルム


In [8]:
import re
import pandas as pd
import dateparser
from utils.logger import get_logger

logger = get_logger(__name__)


def extract_and_format_death_info(text):
    """
    _data_processor.py の _extract_text_from_cell メソッドで抽出されたテキストから、
    key_type の "death_date" に基づいて、死亡年月日、年齢、死亡地の情報を詳細に抽出し、DataFrame を作成する。

    Args:
        text (str): 処理対象のテキスト（例: "1955-04-18  1955年 4月18日 76歳没 アメリカ合衆国 ニュージャージー州 プリンストン"）

    Returns:
        pandas.DataFrame: 死亡年月日、年齢、死亡地の情報を格納した DataFrame。
                          テキストから情報が抽出できなかった場合は空の DataFrame を返す。
    """
    logger.debug(f"extract_and_format_death_info: 処理対象テキスト: {text}")

    # 日付の抽出 (YYYY年 MM月 DD日 または YYYY-MM-DD)
    date_match = re.search(r"(\d{4})年\s*(\d{1,2})月(\d{1,2})日|(\d{4})-(\d{1,2})-(\d{1,2})", text)
    if date_match:
        if date_match.group(1):
            year_str, month_str, day_str = date_match.groups()[:3]
            date_str = f"{year_str}年{month_str}月{day_str}日"
        else:
            year_str, month_str, day_str = date_match.groups()[3:]
            date_str = f"{year_str}年{month_str}月{day_str}日"

    else:
        date_str = None

    # 死亡年齢の抽出
    age_match = re.search(r"(\d+)歳没", text)
    age_str = age_match.group(1) if age_match else None

    # 死亡地の抽出
    place_match = re.search(r"(.+)$", text)
    if place_match:
        place_str = place_match.group(1)
        # 日付と年齢を削除
        place_str = re.sub(r"(\d{4})年\s*(\d{1,2})月(\d{1,2})日|(\d{4})-(\d{1,2})-(\d{1,2})|\d+歳没", "", place_str).strip()
    else:
        place_str = None

    if not date_str:
        logger.warning("extract_and_format_death_info: 死亡年月日の抽出に失敗しました。")
        if age_str:
            logger.warning(f"extract_and_format_death_info: 死亡年齢: {age_str}")
        if place_str:
            logger.warning(f"extract_and_format_death_info: 死亡地: {place_str}")
        return pd.DataFrame()

    logger.debug(f"extract_and_format_death_info: 死亡年月日 (抽出): {date_str}")
    if age_str:
        logger.debug(f"extract_and_format_death_info: 死亡年齢: {age_str}")
    if place_str:
        logger.debug(f"extract_and_format_death_info: 死亡地: {place_str}")

    # 日付のフォーマットを統一し、年、月、日に分解
    parsed_date = dateparser.parse(date_str, date_formats=['%Y年%m月%d日'])
    if parsed_date:
        death_year = parsed_date.year
        death_month = parsed_date.month
        death_day = parsed_date.day
    else:
        logger.warning("extract_and_format_death_info: 死亡年月日の解析に失敗しました。")
        return pd.DataFrame()

    logger.debug(f"extract_and_format_death_info: 死亡年: {death_year}, 死亡月: {death_month}, 死亡日: {death_day}")

    # 死亡地の分解
    death_country, death_state, death_city = None, None, None

    if place_str:
        parts = place_str.split()
        if len(parts) >= 3:
            death_country = parts[0]
            death_state = parts[1]
            death_city = parts[2]
        elif len(parts) >= 2:
            death_country = parts[0]
            death_state = parts[1]
            death_city = None
        elif len(parts) == 1:
            death_city = parts[0]
        logger.debug(f"extract_and_format_death_info: 死亡国: {death_country}, 死亡州: {death_state}, 死亡都市: {death_city}")

    # DataFrame の作成
    data = {
        "death_year": [death_year],
        "death_month": [death_month],
        "death_day": [death_day],
        "death_age": [int(age_str) if age_str else None],
        "death_country": [death_country],
        "death_state": [death_state],
        "death_city": [death_city],
    }
    df = pd.DataFrame(data)
    logger.debug(f"extract_and_format_death_info: 作成したDataFrame:\n{df}")

    return df

# テスト用のテキスト
text = "1955-04-18  1955年 4月18日 76歳没 アメリカ合衆国 ニュージャージー州 プリンストン"
# テスト用のテキスト2(都市のみ)
text2 = "1955-04-18  1955年 4月18日 76歳没 プリンストン"
# テスト用のテキスト3(州のみ)
text3 = "1955-04-18  1955年 4月18日 76歳没 ニュージャージー州"
# テスト用のテキスト4(日付なし)
text4 = "76歳没 アメリカ合衆国 ニュージャージー州 プリンストン"

# 関数を実行
df = extract_and_format_death_info(text)
print("test1")
print(df)

df2 = extract_and_format_death_info(text2)
print("test2")
print(df2)

df3 = extract_and_format_death_info(text3)
print("test3")
print(df3)

df4 = extract_and_format_death_info(text4)
print("test4")
print(df4)

2025-03-14 11:19:20.284 | DEBUG    | __main__:extract_and_format_death_info:21 - extract_and_format_death_info: 処理対象テキスト: 1955-04-18  1955年 4月18日 76歳没 アメリカ合衆国 ニュージャージー州 プリンストン
2025-03-14 11:19:20.285 | DEBUG    | __main__:extract_and_format_death_info:57 - extract_and_format_death_info: 死亡年月日 (抽出): 1955年04月18日
2025-03-14 11:19:20.287 | DEBUG    | __main__:extract_and_format_death_info:59 - extract_and_format_death_info: 死亡年齢: 76
2025-03-14 11:19:20.288 | DEBUG    | __main__:extract_and_format_death_info:61 - extract_and_format_death_info: 死亡地: アメリカ合衆国 ニュージャージー州 プリンストン
2025-03-14 11:19:20.288 | DEBUG    | __main__:extract_and_format_death_info:73 - extract_and_format_death_info: 死亡年: 1955, 死亡月: 4, 死亡日: 18
2025-03-14 11:19:20.289 | DEBUG    | __main__:extract_and_format_death_info:90 - extract_and_format_death_info: 死亡国: アメリカ合衆国, 死亡州: ニュージャージー州, 死亡都市: プリンストン
2025-03-14 11:19:20.291 | DEBUG    | __main__:extract_and_format_death_info:103 - extract_and_format_death_info: 作成したDataFrame:
   

test1
   death_year  death_month  death_day  death_age  death_country      death_state       death_city 
0     1955          4          18         76      アメリカ合衆国  ニュージャージー州  プリンストン
test2
   death_year  death_month  death_day  death_age death_country death_state   death_city 
0     1955          4          18         76          None         None     プリンストン
test3
   death_year  death_month  death_day  death_age death_country death_state      death_city    
0     1955          4          18         76          None         None     ニュージャージー州
test4
Empty DataFrame
Columns: []
Index: []


In [2]:
import re

text = """
アインシュタインは1879年3月14日、 ヘルマン・アインシュタイン を父、 パウリーネ・コッホ を母とし、その長男としてドイツ南西部の バーデンヴュルテンベルク州 ウルム市 にて生まれた 。父ヘルマンはその弟ヤコブから誘われ、アルベルト誕生翌年の1880年夏、一家は ミュンヘン に引っ越し、兄弟は、直流電流に基づいた電気機器を製造する会社Elektrotechnische Fabrik J Einstein Cieを設立した。ヘルマンは営業を担当しヤコブは技術を担当した。1881年には一家にマリアアルベルトの妹。通称マーヤ。が誕生し、一家は1894年まで同地ミュンヘンで暮らすことになる。 アインシュタインは、5歳ごろまであまり言葉を発して他人と会話することがなかった 。しかし、5歳のときに父親からもらった 方位磁針 が、自然界の仕組みに対する興味をもたらすきっかけとなった 。また、同じ頃、 ヴァイオリン を習い始めている 。そしてすぐに モーツァルト の曲が好きになり、ヴァイオリンは生涯の友となった。 アインシュタイン一家はその家系からして アシュケナージ系ユダヤ人 ではあったものの、敬虔なユダヤ教徒というわけではなかったため、アインシュタインは5歳から3年間、ミュンヘンにある カトリック 系の公立学校へ通った。卒業後はミュンヘンの ルイトポルト・ギムナジウム 英語版 現在ではアルバート・アインシュタイン・ギムナジウムと呼ばれている学校に入学。以後7年間、ドイツを離れイタリアに行くまで教育を受ける。しかし、同校の 軍国主義 的で重苦しい校風にはなじめなかった 。 幼少のころは、言葉を理解したり話したりするという面では問題がなかったが、言葉を出すのには時間を要した。一方で数学に関しては傑出した才能を示し、9歳のときに ピタゴラスの定理 の存在を知り、その定理の美しい証明を寝る間も惜しんで考え、そして自力で定理を証明した。12歳のときに家庭教師の マックス・タルムード から、 テオドール・シュピーカー の ユークリッド幾何学 の本をもらい 独習 。 微分 学と 積分 学も、この当時に独学で習得したといわれている。同じころ、医学生だった マックス・タルメイ から 天文学 の存在を知らされ、同時に物理学に関心を示すようになったという。 1894年 、父と叔父の会社が行きづまり 、その結果、新たな商業的な機会を求めて一家は イタリア の ミラノ に引っ越すことになった。父ヘルマンはアインシュタインがギムナジウムをしっかり卒業する必要があると判断し、アインシュタインだけ同地に残されることになった。父ヘルマンはアルベルトが 電気工学 の道へと進むといいと考えていたのだった。だがアインシュタインは規則ずくめで軍国主義的な校風と対立・反発し、1894年12月末、医師に書かせた診断書を口実にして退校を申し出て、家族を追って旅をし、当時イタリアの パヴィア にいた家族のもとへとやってきた 。このイタリアでの滞在中、アインシュタインは磁界中での エーテル の状態の調査についてという題名の短い試論を書いたという。
"""

def extract_parents_info(text):
    father_pattern = re.compile(r'(?P<father>[\w・ー]+)\s*を父')
    mother_pattern = re.compile(r'(?P<mother>[\w・ー]+)\s*を母')

    father_match = father_pattern.search(text)
    mother_match = mother_pattern.search(text)

    father_name = father_match.group('father') if father_match else None
    mother_name = mother_match.group('mother') if mother_match else None

    return {
        'father': father_name,
        'mother': mother_name
    }

parents_info = extract_parents_info(text)
print(parents_info)

{'father': 'ヘルマン・アインシュタイン', 'mother': 'パウリーネ・コッホ'}


In [1]:
import re

# 入力データ
data = {"配偶者": "ミレヴァ・マリッチ 1903-1919 エルザ・レーベンタール 1919-1936"}

# 配偶者情報を抽出
spouse_info = data["配偶者"]

# 正規表現で氏名、結婚年、離婚年を抽出
pattern = r"([^\d]+) (\d{4})-(\d{4})"
matches = re.findall(pattern, spouse_info)

# 結果をリストに格納
result = []
for match in matches:
    result.append({"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": int(match[2])})

# 結果を表示
print(result)

[{'氏名': 'ミレヴァ・マリッチ', '結婚年': 1903, '離婚年': 1919}, {'氏名': 'エルザ・レーベンタール', '結婚年': 1919, '離婚年': 1936}]


In [2]:
import re

# 入力データ
data = {"配偶者": "ミレヴァ・マリッチ 1903-1919 エルザ・レーベンタール 1919-1936 ピエール・キュリー 1895年結婚"}

# 配偶者情報を抽出
spouse_info = data["配偶者"]

# 正規表現で氏名、結婚年、離婚年を抽出
pattern_full = r"([^\d]+) (\d{4})-(\d{4})"
pattern_only_marriage = r"([^\d]+) (\d{4})年結婚"
matches_full = re.findall(pattern_full, spouse_info)
matches_only_marriage = re.findall(pattern_only_marriage, spouse_info)

# 結果をリストに格納
result = []
for match in matches_full:
    result.append({"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": int(match[2])})

for match in matches_only_marriage:
    result.append({"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": None})

# 結果を表示
print(result)

[{'氏名': 'ミレヴァ・マリッチ', '結婚年': 1903, '離婚年': 1919}, {'氏名': 'エルザ・レーベンタール', '結婚年': 1919, '離婚年': 1936}, {'氏名': 'ピエール・キュリー', '結婚年': 1895, '離婚年': None}]


In [3]:
import re

# 入力データ
data = {"配偶者": "ミレヴァ・マリッチ 1903-1919 エルザ・レーベンタール 1919-1936 ピエール・キュリー 1895年結婚"}

# 配偶者情報を抽出
spouse_info = data["配偶者"]

# 正規表現で氏名、結婚年、離婚年を抽出
pattern_full = r"([^\d]+) (\d{4})-(\d{4})"
pattern_only_marriage = r"([^\d]+) (\d{4})\s*年\s*結婚"
matches_full = re.findall(pattern_full, spouse_info)
matches_only_marriage = re.findall(pattern_only_marriage, spouse_info)

# 結果をリストに格納
result = []
for match in matches_full:
    result.append({"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": int(match[2])})

for match in matches_only_marriage:
    result.append({"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": None})

# 結果を表示
print(result)

[{'氏名': 'ミレヴァ・マリッチ', '結婚年': 1903, '離婚年': 1919}, {'氏名': 'エルザ・レーベンタール', '結婚年': 1919, '離婚年': 1936}, {'氏名': 'ピエール・キュリー', '結婚年': 1895, '離婚年': None}]


In [4]:
import re

# 入力データ
data = {"配偶者": "ミレヴァ・マリッチ 1903-1919 エルザ・レーベンタール 1919-1936 ピエール・キュリー 1895年結婚 ローレン・パウエル・ジョブズ"}

# 配偶者情報を抽出
spouse_info = data["配偶者"]

# 正規表現パターンのリスト
patterns = [
    (r"([^\d]+) (\d{4})-(\d{4})", lambda match: {"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": int(match[2])}),
    (r"([^\d]+) (\d{4})\s*年\s*結婚", lambda match: {"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": None}),
    (r"([^\d]+)(?!\s*\d{4})", lambda match: {"氏名": match[0].strip(), "結婚年": None, "離婚年": None})
]

# 結果をリストに格納
result = []
for pattern, func in patterns:
    matches = re.findall(pattern, spouse_info)
    result.extend([func(match) for match in matches])
    # マッチした部分を削除して重複を防ぐ
    spouse_info = re.sub(pattern, '', spouse_info)

# 結果を表示
print(result)

[{'氏名': 'ミレヴァ・マリッチ', '結婚年': 1903, '離婚年': 1919}, {'氏名': 'エルザ・レーベンタール', '結婚年': 1919, '離婚年': 1936}, {'氏名': 'ピエール・キュリー', '結婚年': 1895, '離婚年': None}, {'氏名': '', '結婚年': None, '離婚年': None}]


In [5]:
import re

# 入力データ
data = {"配偶者": "ミレヴァ・マリッチ 1903-1919 エルザ・レーベンタール 1919-1936 ピエール・キュリー 1895年結婚 ローレン・パウエル・ジョブズ"}

# 家族構成データ
family_structure = [
    {"関係": "母", "氏名": "パウリーネ・コッホ"},
    {"関係": "父", "氏名": "ヘルマン・アインシュタイン"},
    {"関係": "妹", "氏名": "マヤ・アインシュタイン"},
    {"関係": "1人目の妻", "氏名": "ミレヴァ・マリッチ"},
    {"関係": "2人目の妻", "氏名": "エルザ・アインシュタイン"},
    {"関係": "娘", "氏名": "リーゼル・アインシュタイン"},
    {"関係": "息子", "氏名": "ハンス・アルベルト・アインシュタイン"},
    {"関係": "息子", "氏名": "エドゥアルト・アインシュタイン"},
    {"関係": "孫", "氏名": "ベルンハルト・アインシュタイン"},
    {"関係": "孫", "氏名": "イヴリン・アインスタイン"}
]

# 配偶者情報を抽出
spouse_info = data["配偶者"]

# 正規表現パターンのリスト
patterns = [
    # 結婚年と離婚年がある場合
    (r"([^\d]+?) (\d{4})-(\d{4})", lambda match: {"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": int(match[2])}),
    # 結婚年のみがある場合
    (r"([^\d]+?) (\d{4})\s*年\s*結婚", lambda match: {"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": None}),
    # 年数が不明な場合
    (r"([^\d]+?)(?=\s|$)", lambda match: {"氏名": match[0].strip(), "結婚年": None, "離婚年": None})
]

# 配偶者の情報を格納するリスト
spouses = []

# パターンに基づいて配偶者情報を抽出
try:
    for pattern, func in patterns:
        matches = re.findall(pattern, spouse_info)
        spouses.extend([func(match) for match in matches])
        # マッチした部分を削除して重複を防ぐ
        spouse_info = re.sub(pattern, '', spouse_info)
except Exception as e:
    print(f"エラーが発生しました: {e}")

# 家族構成にデータを追加
for spouse in spouses:
    for member in family_structure:
        if member["氏名"] == spouse["氏名"]:
            member["結婚年"] = spouse["結婚年"]
            member["離婚年"] = spouse["離婚年"]

# 結果を表示
import json
print(json.dumps(family_structure, ensure_ascii=False, indent=2))

[
  {
    "関係": "母",
    "氏名": "パウリーネ・コッホ"
  },
  {
    "関係": "父",
    "氏名": "ヘルマン・アインシュタイン"
  },
  {
    "関係": "妹",
    "氏名": "マヤ・アインシュタイン"
  },
  {
    "関係": "1人目の妻",
    "氏名": "ミレヴァ・マリッチ",
    "結婚年": 1903,
    "離婚年": 1919
  },
  {
    "関係": "2人目の妻",
    "氏名": "エルザ・アインシュタイン"
  },
  {
    "関係": "娘",
    "氏名": "リーゼル・アインシュタイン"
  },
  {
    "関係": "息子",
    "氏名": "ハンス・アルベルト・アインシュタイン"
  },
  {
    "関係": "息子",
    "氏名": "エドゥアルト・アインシュタイン"
  },
  {
    "関係": "孫",
    "氏名": "ベルンハルト・アインシュタイン"
  },
  {
    "関係": "孫",
    "氏名": "イヴリン・アインスタイン"
  }
]


In [5]:
import re

# 入力データ
data = {"配偶者": "ミレヴァ・マリッチ 1903-1919 エルザ・レーベンタール 1919-1936 ピエール・キュリー 1895年結婚 ローレン・パウエル・ジョブズ"}

# 同一人物の名前の統一マッピング
name_mapping = {
    "エルザ・レーベンタール": "エルザ・アインシュタイン"
}

# 家族構成データ
family_structure = [
    {"関係": "母", "氏名": "パウリーネ・コッホ"},
    {"関係": "父", "氏名": "ヘルマン・アインシュタイン"},
    {"関係": "妹", "氏名": "マヤ・アインシュタイン"},
    {"関係": "1人目の妻", "氏名": "ミレヴァ・マリッチ"},
    {"関係": "2人目の妻", "氏名": "エルザ・アインシュタイン"},
    {"関係": "娘", "氏名": "リーゼル・アインシュタイン"},
    {"関係": "息子", "氏名": "ハンス・アルベルト・アインシュタイン"},
    {"関係": "息子", "氏名": "エドゥアルト・アインシュタイン"},
    {"関係": "孫", "氏名": "ベルンハルト・アインシュタイン"},
    {"関係": "孫", "氏名": "イヴリン・アインスタイン"}
]

# 配偶者情報を抽出
spouse_info = data["配偶者"]

# 正規表現パターンのリスト
patterns = [
    # 結婚年と離婚年がある場合
    (r"([^\d]+?) (\d{4})-(\d{4})", lambda match: {"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": int(match[2])}),
    # 結婚年のみがある場合
    (r"([^\d]+?) (\d{4})\s*年\s*結婚", lambda match: {"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": None}),
    # 年数が不明な場合
    (r"([^\d]+?)(?=\s|$)", lambda match: {"氏名": match[0].strip(), "結婚年": None, "離婚年": None})
]

# 配偶者の情報を格納するリスト
spouses = []

# パターンに基づいて配偶者情報を抽出
try:
    for pattern, func in patterns:
        matches = re.findall(pattern, spouse_info)
        spouses.extend([func(match) for match in matches])
        # マッチした部分を削除して重複を防ぐ
        spouse_info = re.sub(pattern, '', spouse_info)
except Exception as e:
    print(f"エラーが発生しました: {e}")

# 家族構成にデータを追加
for spouse in spouses:
    # 同一人物の名前を統一
    if spouse["氏名"] in name_mapping:
        spouse["氏名"] = name_mapping[spouse["氏名"]]
    
    for member in family_structure:
        if member["氏名"] == spouse["氏名"]:
            member["結婚年"] = spouse["結婚年"]
            member["離婚年"] = spouse["離婚年"]

# 結果を表示
import json
print(json.dumps(family_structure, ensure_ascii=False, indent=2))


--- 自動マッピング候補の検出 ---
情報: 'ミレヴァ・マリッチ' は family_structure に既に存在します。マッピング不要。
候補検出: 'エルザ・レーベンタール' -> 'エルザ・アインシュタイン' (理由: 最初の部分 'エルザ' が一致)
情報: 'ピエール・キュリー' は family_structure 内に一致する候補が見つかりませんでした。
情報: '年結婚' は family_structure 内に一致する候補が見つかりませんでした。
情報: 'ローレン・パウエル・ジョブズ' は family_structure 内に一致する候補が見つかりませんでした。

--- 家族構成への情報統合 ---
更新: 'ミレヴァ・マリッチ' に 結婚年 1903 を追加しました。
更新: 'ミレヴァ・マリッチ' に 離婚年 1919 を追加しました。
更新: 'エルザ・アインシュタイン' に 結婚年 1919 を追加しました。
更新: 'エルザ・アインシュタイン' に 離婚年 1936 を追加しました。
情報: 抽出された配偶者 'ピエール・キュリー' は現在の family_structure には見つかりませんでした。
情報: 抽出された配偶者 '年結婚' は現在の family_structure には見つかりませんでした。
情報: 抽出された配偶者 'ローレン・パウエル・ジョブズ' は現在の family_structure には見つかりませんでした。

--- 最終的な家族構成 ---
[
  {
    "関係": "母",
    "氏名": "パウリーネ・コッホ"
  },
  {
    "関係": "父",
    "氏名": "ヘルマン・アインシュタイン"
  },
  {
    "関係": "妹",
    "氏名": "マヤ・アインシュタイン"
  },
  {
    "関係": "1人目の妻",
    "氏名": "ミレヴァ・マリッチ",
    "結婚年": 1903,
    "離婚年": 1919
  },
  {
    "関係": "2人目の妻",
    "氏名": "エルザ・アインシュタイン",
    "結婚年": 1919,
    "離婚年": 1936
  },
  {
    "関係

In [7]:
import re
import json

# --- 入力データと既存の構造 ---
# 入力データ
data = {"配偶者": "ミレヴァ・マリッチ 1903-1919 エルザ・レーベンタール 1919-1936 ピエール・キュリー 1895年結婚 ローレン・パウエル・ジョブズ"}

# 家族構成データ (初期状態)
family_structure = [
    {"関係": "母", "氏名": "パウリーネ・コッホ"},
    {"関係": "父", "氏名": "ヘルマン・アインシュタイン"},
    {"関係": "妹", "氏名": "マヤ・アインシュタイン"},
    {"関係": "1人目の妻", "氏名": "ミレヴァ・マリッチ"},
    {"関係": "2人目の妻", "氏名": "エルザ・アインシュタイン"}, # 正準名
    {"関係": "娘", "氏名": "リーゼル・アインシュタイン"},
    {"関係": "息子", "氏名": "ハンス・アルベルト・アインシュタイン"},
    {"関係": "息子", "氏名": "エドゥアルト・アインシュタイン"},
    {"関係": "孫", "氏名": "ベルンハルト・アインシュタイン"},
    {"関係": "孫", "氏名": "イヴリン・アインスタイン"}
    # ピエール・キュリーやローレン・パウエル・ジョブズはアインシュタインの家族構成外なので、初期リストには含めない
]

# --- 配偶者情報の抽出 (既存ロジック) ---
spouse_info = data["配偶者"]
patterns = [
    (r"([^\d]+?) (\d{4})-(\d{4})", lambda match: {"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": int(match[2])}),
    (r"([^\d]+?) (\d{4})\s*年\s*結婚", lambda match: {"氏名": match[0].strip(), "結婚年": int(match[1]), "離婚年": None}),
    (r"([^\d]+?)(?=\s|$)", lambda match: {"氏名": match[0].strip(), "結婚年": None, "離婚年": None})
]
spouses_extracted = []
temp_spouse_info = spouse_info # 元の文字列を変更しないようにコピー
try:
    for pattern, func in patterns:
        # finditerを使ってマッチした位置も考慮し、重複を防ぐ改良
        processed_indices = set()
        for match in re.finditer(pattern, temp_spouse_info):
            # マッチ範囲が既に処理された部分と重なっていないか確認
            is_overlapping = False
            for start, end in processed_indices:
                if max(match.start(), start) < min(match.end(), end):
                    is_overlapping = True
                    break
            if not is_overlapping:
                spouses_extracted.append(func(match.groups()))
                processed_indices.add((match.start(), match.end()))

        # findallを使う場合（元のロジックに近いが、重複の可能性あり）
        # matches = re.findall(pattern, temp_spouse_info)
        # spouses_extracted.extend([func(match) for match in matches])
        # temp_spouse_info = re.sub(pattern, '', temp_spouse_info) # この削除が意図しない結果を招くことがある

except Exception as e:
    print(f"配偶者情報の抽出中にエラーが発生しました: {e}")

# 重複する可能性のある名前を整理 (例: 年数ありとなしで同じ名前が抽出された場合)
unique_spouses = {}
for s in spouses_extracted:
    name = s['氏名']
    if name not in unique_spouses:
        unique_spouses[name] = s
    else:
        # より情報が多い方を優先 (例: 年数がある方)
        if s.get('結婚年') is not None and unique_spouses[name].get('結婚年') is None:
            unique_spouses[name] = s
        elif s.get('離婚年') is not None and unique_spouses[name].get('離婚年') is None:
            unique_spouses[name]['離婚年'] = s['離婚年'] # 離婚年だけ更新

spouses = list(unique_spouses.values())

# --- 自動での名前マッピング生成ロジック ---

def get_first_part(name):
    """名前の最初の部分（スペースまたは・の前）を取得"""
    # 全角スペースも考慮
    parts = re.split(r'[ ・\u3000]', name, 1)
    return parts[0]

# family_structureから正準名のリストとセットを作成
canonical_names = [member["氏名"] for member in family_structure]
canonical_names_set = set(canonical_names)

# 抽出された配偶者名を取得
input_names = [spouse["氏名"] for spouse in spouses]

# 自動マッピング辞書を作成
auto_name_mapping = {}

print("\n--- 自動マッピング候補の検出 ---")
for input_name in input_names:
    if input_name not in canonical_names_set:
        input_first_part = get_first_part(input_name)
        if not input_first_part: # 名前が空などの場合スキップ
            continue

        found_match = False
        for canonical_name in canonical_names:
            canonical_first_part = get_first_part(canonical_name)
            # 最初の部分が一致し、かつ空文字列でない場合
            if input_first_part == canonical_first_part and input_first_part:
                # 既に見つかっている場合、曖昧さの警告（必要に応じて）
                if input_name in auto_name_mapping:
                    print(f"警告: '{input_name}' は複数の候補 ('{auto_name_mapping[input_name]}', '{canonical_name}') にマッチする可能性があります。最初のマッチを採用します。")
                else:
                    print(f"候補検出: '{input_name}' -> '{canonical_name}' (理由: 最初の部分 '{input_first_part}' が一致)")
                    auto_name_mapping[input_name] = canonical_name
                    found_match = True
                    break # 最初のマッチを採用

        if not found_match:
            print(f"情報: '{input_name}' は family_structure 内に一致する候補が見つかりませんでした。")
    else:
        print(f"情報: '{input_name}' は family_structure に既に存在します。マッピング不要。")


# --- 家族構成へのデータ統合 ---
print("\n--- 家族構成への情報統合 ---")
for spouse in spouses:
    original_name = spouse["氏名"]
    # 自動生成されたマッピングを使用
    mapped_name = auto_name_mapping.get(original_name, original_name) # マッピングがあれば使い、なければ元の名前

    found_in_family = False
    for member in family_structure:
        if member["氏名"] == mapped_name:
            # 年情報が存在すれば更新
            if spouse.get("結婚年") is not None:
                member["結婚年"] = spouse["結婚年"]
                print(f"更新: '{mapped_name}' に 結婚年 {spouse['結婚年']} を追加しました。")
            if spouse.get("離婚年") is not None:
                member["離婚年"] = spouse["離婚年"]
                print(f"更新: '{mapped_name}' に 離婚年 {spouse['離婚年']} を追加しました。")
            found_in_family = True
            break # 対応するメンバーを見つけたら次へ

    # family_structure に対応する人物が見つからなかった場合 (例: ピエール・キュリー)
    if not found_in_family and original_name == mapped_name: # マッピングもされなかった場合
        print(f"情報: 抽出された配偶者 '{original_name}' は現在の family_structure には見つかりませんでした。")
        # 必要であれば、新しいメンバーとして追加するロジックをここに追加することも可能
        # family_structure.append({
        #     "関係": "不明な関係の配偶者", # 関係を特定する必要あり
        #     "氏名": original_name,
        #     "結婚年": spouse.get("結婚年"),
        #     "離婚年": spouse.get("離婚年")
        # })


# --- 結果表示 ---
print("\n--- 最終的な家族構成 ---")
print(json.dumps(family_structure, ensure_ascii=False, indent=2))


--- 自動マッピング候補の検出 ---
情報: 'ミレヴァ・マリッチ' は family_structure に既に存在します。マッピング不要。
候補検出: 'エルザ・レーベンタール' -> 'エルザ・アインシュタイン' (理由: 最初の部分 'エルザ' が一致)
情報: 'ピエール・キュリー' は family_structure 内に一致する候補が見つかりませんでした。
情報: '年結婚' は family_structure 内に一致する候補が見つかりませんでした。
情報: 'ローレン・パウエル・ジョブズ' は family_structure 内に一致する候補が見つかりませんでした。

--- 家族構成への情報統合 ---
更新: 'ミレヴァ・マリッチ' に 結婚年 1903 を追加しました。
更新: 'ミレヴァ・マリッチ' に 離婚年 1919 を追加しました。
更新: 'エルザ・アインシュタイン' に 結婚年 1919 を追加しました。
更新: 'エルザ・アインシュタイン' に 離婚年 1936 を追加しました。
情報: 抽出された配偶者 'ピエール・キュリー' は現在の family_structure には見つかりませんでした。
情報: 抽出された配偶者 '年結婚' は現在の family_structure には見つかりませんでした。
情報: 抽出された配偶者 'ローレン・パウエル・ジョブズ' は現在の family_structure には見つかりませんでした。

--- 最終的な家族構成 ---
[
  {
    "関係": "母",
    "氏名": "パウリーネ・コッホ"
  },
  {
    "関係": "父",
    "氏名": "ヘルマン・アインシュタイン"
  },
  {
    "関係": "妹",
    "氏名": "マヤ・アインシュタイン"
  },
  {
    "関係": "1人目の妻",
    "氏名": "ミレヴァ・マリッチ",
    "結婚年": 1903,
    "離婚年": 1919
  },
  {
    "関係": "2人目の妻",
    "氏名": "エルザ・アインシュタイン",
    "結婚年": 1919,
    "離婚年": 1936
  },
  {
    "関係